# Basic ERDDAP Obs Matching

This notebook is a demonstration of a basic recommended method to collect SalishSeaCast model field values to match
ocean observations.
It uses `xarray` to access the SalishSeaCast model datasets via the ERDDAP server.
The observations are assumed to be defined by a collection of 4d time-space coordinates in a `pandas.DataFrame`.
(Those could, of course, be read from a CSV file.)
The model field values closest to the observations are combined with the observations to create a consolidated
`pandas.DataFrame`.


It is assumed that the observations are discrete and independent;
i.e. single points in time-space.

Extraction of time series, depth profiles, or other hyperslabs of model fields would need to be done differently.
For members of the MOAD group who have access to the model results file storage those things are best done using
[Reshapr](https://reshapr.readthedocs.io/en/latest/).

Collection of model field values to match observations using the method demonstrated here is
a process that lends itself well to parallelization,
in contrast to the simple loop iteration that is used in this notebook.
Scaling the method demonstrated here to match hundreds or thousands of observations is the subject of
another notebook in this directory.

The conda environment file used to run this notebook is `environment.yaml` in this directory.

In [1]:
import pandas as pd
import xarray as xr

For reference, the versions of Python and the packages that are important to what we are doing
that were in the environment in which this notebook was last run are:

In [2]:
import sys
import netCDF4
import numpy
import pandas
import xarray

print(f"Python {sys.version=}")
print(f"{numpy.__version__=}")
print(f"{xarray.__version__=}")
print(f"{pandas.__version__=}")
print(f"{netCDF4.__version__=}")

Python sys.version='3.13.2 | packaged by conda-forge | (main, Feb 17 2025, 14:10:22) [GCC 13.3.0]'
numpy.__version__='2.2.3'
xarray.__version__='2025.1.2'
pandas.__version__='2.2.3'
netCDF4.__version__='1.7.2'


## Basic ERDDAP Dataset Access with `xarray`

* The SalishSeaCast ERDDAP server is at https://salishsea.eos.ubc.ca/erddap.
* The list of datasets published by the server is at https://salishsea.eos.ubc.ca/erddap/info/index.html.
  The dataset ids are in the rightmost column of the table on that page.
* You can look at the metadata for a specific dataset by clicking the `M` link in the "FGDC, ISO, Metadata" column of the table.
  That will take you to a page with a URL of the form `https://salishsea.eos.ubc.ca/erddap/info/dataset_id/index.html`,
  for example: https://salishsea.eos.ubc.ca/erddap/info/ubcSSg3DPhysicsFields1hV21-11/index.html
* To access that dataset using `xarray.open_dataset()`,
  use a URL of the form `https://salishsea.eos.ubc.ca/erddap/griddap/dataset_id`

In [3]:
erddap_url = "https://salishsea.eos.ubc.ca/erddap"
dataset_id = "ubcSSg3DPhysicsFields1hV21-11"
dataset_url = f"{erddap_url}/griddap/{dataset_id}"

In [4]:
ds = xr.open_dataset(dataset_url)

ds

<xarray.Dataset> Size: 27TB
Dimensions:      (time: 159408, depth: 40, gridY: 898, gridX: 398)
Coordinates:
  * time         (time) datetime64[ns] 1MB 2007-01-01T00:30:00 ... 2025-03-08...
  * depth        (depth) float32 160B 0.5 1.5 2.5 3.5 ... 387.6 414.5 441.5
  * gridY        (gridY) int16 2kB 0 1 2 3 4 5 6 ... 891 892 893 894 895 896 897
  * gridX        (gridX) int16 796B 0 1 2 3 4 5 6 ... 392 393 394 395 396 397
Data variables:
    salinity     (time, depth, gridY, gridX) float32 9TB ...
    temperature  (time, depth, gridY, gridX) float32 9TB ...
    sigma_theta  (time, depth, gridY, gridX) float32 9TB ...
Attributes: (12/25)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2025-03-08T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2025-Mar-08 21:23:58 GMT
    title:                     Green, Salish Sea, 3d Physics Fields, Hourly, ...
    uuid:                      0a903001-1231-450e-97f3-d094561d5239

`xarray` defers loading the actual values of the fields in the dataset as long as possible.
That's known as lazy-loading.
The information returned by `xarray.open_data()` in the cell above is just the dataset metadata.

The ERDDAP server has 2 important limits that control how much data you can retrieve in a single operation:

1. The maximum size of the data transfer must be less than 2 Gb
2. The maximum processing time on the server to extract the data from the underlying files must be less than 10 minutes

So, to get the maximum amount of useful data within those limits, we need to limit our data requests in every way possible.
The first way to do that is to reduce the set of variables we are requesting from the server to only those we are interested in.
Subtracting the set of variables that we are interested in from the set of all variables in the dataset gives us a set of variables
that we can tell the server to drop from our request:


In [5]:
all_vars = set(ds.data_vars)
keep_vars = {"salinity", "temperature"}
drop_vars = all_vars - keep_vars
ds = xr.open_dataset(dataset_url, drop_variables=drop_vars)

ds

<xarray.Dataset> Size: 18TB
Dimensions:      (time: 159408, depth: 40, gridY: 898, gridX: 398)
Coordinates:
  * time         (time) datetime64[ns] 1MB 2007-01-01T00:30:00 ... 2025-03-08...
  * depth        (depth) float32 160B 0.5 1.5 2.5 3.5 ... 387.6 414.5 441.5
  * gridY        (gridY) int16 2kB 0 1 2 3 4 5 6 ... 891 892 893 894 895 896 897
  * gridX        (gridX) int16 796B 0 1 2 3 4 5 6 ... 392 393 394 395 396 397
Data variables:
    salinity     (time, depth, gridY, gridX) float32 9TB ...
    temperature  (time, depth, gridY, gridX) float32 9TB ...
Attributes: (12/25)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2025-03-08T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2025-Mar-08 21:23:58 GMT
    title:                     Green, Salish Sea, 3d Physics Fields, Hourly, ...
    uuid:                      0a903001-1231-450e-97f3-d094561d5239

## Matching a Single Observation Point

Let's consider a single salinity observation for which we want to get the nearest model salinity field value.
If we read that observation from a CSV file into a `pandas.DataFrame` it would look like:

In [6]:
obs_df = pd.DataFrame(
    {
        "time": [pd.to_datetime("2025-03-07 11:45:00")],
        "depth": [0],
        "gridY": [350],
        "gridX": [250],
        "salinity": [28.5],
        "temperature": [7.5],
    }
)

obs_df

,time,depth,gridY,gridX,salinity,temperature
0,2025-03-07 11:45:00,0,350,250,28.5,7.5


The time and depth values of the observation are unlikely to exactly match those of a model calculation.
So, we use the `method="nearest"` argument to do a "nearest neighbour" lookup on both of those coordinates.

In [7]:
ds_pt = (ds
    .sel(time=obs_df["time"][0], method="nearest")
    .sel(depth=obs_df["depth"][0], method="nearest")
    .sel(gridY=obs_df["gridY"][0], gridX=obs_df["gridX"][0])
)

ds_pt

<xarray.Dataset> Size: 24B
Dimensions:      ()
Coordinates:
    time         datetime64[ns] 8B 2025-03-07T11:30:00
    depth        float32 4B 0.5
    gridY        int16 2B 350
    gridX        int16 2B 250
Data variables:
    salinity     float32 4B ...
    temperature  float32 4B ...
Attributes: (12/25)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2025-03-08T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2025-Mar-08 21:23:58 GMT
    title:                     Green, Salish Sea, 3d Physics Fields, Hourly, ...
    uuid:                      0a903001-1231-450e-97f3-d094561d5239

Note that the coordinate values in the dataset we get from ERDDAP are model coordinate values;
i.e. the nearest values to those that we requested:

In [8]:
ds_pt.time.values, ds_pt.depth.values

(np.datetime64('2025-03-07T11:30:00.000000000'),
 array(0.5000003, dtype=float32))

In [9]:
ds_pt.salinity.values, ds_pt.temperature.values

(array(28.972582, dtype=float32), array(8.001064, dtype=float32))

Accessing the model field values here causes an actual data request,
which takes longer than the lazy-loading metadata requests above.

## Iterating Over Multiple Observations

To process multiple observations, we'll use a list comprehension.
That is faster than a `for` loop iterating over `DataFrame.iterrows()`.
It is possible that `DataFrame.apply()` might be faster, but I was unable
to come up with a function to use in `.apply()` that could return the model
time, depth, and field values.

The first step is to define a function that takes the ERDDAP dataset and the
observation coordinates that we want model field values at.
It will return the model time and depth that are nearest to those of the observation,
as well as the model filed values.
We'll start by just returning the `xarray.Dataset` that results from the `.sel()` operations.

In [10]:
def get_model_values(ds, time, depth, gridY, gridX):
    ds_pt = (ds
        .sel(time=time, method="nearest")
        .sel(depth=depth, method="nearest")
        .sel(gridY=gridY, gridX=gridX)
    )
    return ds_pt

We apply to function to the observations dataframe in a list comprehension.
Calling the `.to_numpy()` method on the dataframe converts each row to a NumPy array.
We unpack the first 4 values in that array into the time, depth, y, and x coordinate values
that our `get_model_value()` function takes to use for the selection.
Note that the order of the coordinate variables in the unpacking must match the column order
in the observations dataframe.
The `*_` on the left of the `in` means "label everything after the 1st 4 values in the NumPy array with the variable name `_`".
`*` is the "take everything" part, and `_` is a Python conventional variable name for values we don't care about.

The result of the list comprehension is a list of `xarray.Dataset` object containing the model values,
one for each observation row.

In [11]:
model_values = [
    get_model_values(ds, time, depth, gridY, gridX)
    for time, depth, gridY, gridX, *_ in obs_df.to_numpy()
]

model_values[0]

<xarray.Dataset> Size: 24B
Dimensions:      ()
Coordinates:
    time         datetime64[ns] 8B 2025-03-07T11:30:00
    depth        float32 4B 0.5
    gridY        int16 2B 350
    gridX        int16 2B 250
Data variables:
    salinity     float32 4B ...
    temperature  float32 4B ...
Attributes: (12/25)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2025-03-08T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2025-Mar-08 21:23:58 GMT
    title:                     Green, Salish Sea, 3d Physics Fields, Hourly, ...
    uuid:                      0a903001-1231-450e-97f3-d094561d5239

Since our observations are in a dataframe,
it is probably most convenient to have our function also return a dataframe.
We label the model values with the prefix `model_` to avoid conflict with the observations column names.
We have to extract the model values from the `xarray.DataArray` objects that they are in.
To preserve the timestamp nature of the model time, we use the `.value` property.
To convert the depth and model variable field values to Python scalars
(the same types as the observation values), we use the `.item()` method.

Now our result is a list of `pandas.DataFrame` objects containing the model values,
one for each observation row.

In [12]:
def get_model_values_df(ds, time, depth, gridY, gridX):
    ds_pt = (ds
        .sel(time=time, method="nearest")
        .sel(depth=depth, method="nearest")
        .sel(gridY=gridY, gridX=gridX)
    )
    df_pt = pd.DataFrame(
        {
            "time": [ds_pt.time.values],
            "depth": [ds_pt.depth.item()],
            "salinity": [ds_pt.salinity.item()],
            "temperature": [ds_pt.temperature.item()],
        }
    )
    return df_pt

In [13]:
model_dfs = [
    get_model_values_df(ds, time, depth, gridY, gridX)
    for time, depth, gridY, gridX, *_ in obs_df.to_numpy()
]

The list comprehension operation above triggers a data access request to the server because of the `.to_numpy()` call.

In [14]:
model_values[0]

<xarray.Dataset> Size: 24B
Dimensions:      ()
Coordinates:
    time         datetime64[ns] 8B 2025-03-07T11:30:00
    depth        float32 4B 0.5
    gridY        int16 2B 350
    gridX        int16 2B 250
Data variables:
    salinity     float32 4B ...
    temperature  float32 4B ...
Attributes: (12/25)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2025-03-08T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2025-Mar-08 21:23:58 GMT
    title:                     Green, Salish Sea, 3d Physics Fields, Hourly, ...
    uuid:                      0a903001-1231-450e-97f3-d094561d5239

We can combine the observations and the model values into a single dataframe using the `.join()` method.
The `rsuffix` argument of `.join()` lets us avoid the column name conflict between the observations and model values dataframe contents.

In [15]:
result_df = obs_df.join(model_dfs[0], rsuffix="_model")

result_df

,time,depth,gridY,gridX,salinity,temperature,time_model,depth_model,salinity_model,temperature_model
0,2025-03-07 11:45:00,0,350,250,28.5,7.5,2025-03-07 11:30:00,0.5,28.972582,8.001064


Before we scale up to more than 1 observation,
let's generalize our `get_mmodel_values_df()` function so that the model variable names are not hard-coded.

In [16]:
def get_model_values_df(ds, time, depth, gridY, gridX):
    ds_pt = (ds
        .sel(time=time, method="nearest")
        .sel(depth=depth, method="nearest")
        .sel(gridY=gridY, gridX=gridX)
    )
    df_pt = pd.DataFrame(
        {
            "time": [ds_pt.time.values],
            "depth": [ds_pt.depth.item()],
        }
    )
    for var in ds_pt.data_vars:
        df_pt[var] = [ds_pt[var].item()]
    return df_pt

In [17]:
model_dfs = [
    get_model_values_df(ds, time, depth, gridY, gridX)
    for time, depth, gridY, gridX, *_ in obs_df.to_numpy()
]

model_dfs[0]

,time,depth,salinity,temperature
0,2025-03-07 11:30:00,0.5,28.972582,8.001064


In [18]:
result_df = obs_df.join(model_dfs[0], rsuffix="_model")

result_df

,time,depth,gridY,gridX,salinity,temperature,time_model,depth_model,salinity_model,temperature_model
0,2025-03-07 11:45:00,0,350,250,28.5,7.5,2025-03-07 11:30:00,0.5,28.972582,8.001064


Scaling up to more than 1 observation adds one more complication.
We have to concatenate the list of model value dataframes into a single dataframe
before we can join that dataframe with the observations.

In [19]:
obs_df = pd.DataFrame(
    {
        "time": [pd.to_datetime(timestamp) for timestamp in ["2025-03-07 11:59:00", "2025-03-07 12:30:00"]],
        "depth": [0.5, 0.5],
        "gridY": [350, 353],
        "gridX": [250, 247],
        "salinity": [28.5, 28.4],
        "temperature": [7.5, 7.6],
    }
)

obs_df

,time,depth,gridY,gridX,salinity,temperature
0,2025-03-07 11:59:00,0.5,350,250,28.5,7.5
1,2025-03-07 12:30:00,0.5,353,247,28.4,7.6


In [23]:
model_dfs = [
    get_model_values_df(ds, time, depth, gridY, gridX)
    for time, depth, gridY, gridX, *_ in obs_df.to_numpy()
]

model_dfs

[                 time  depth   salinity  temperature
 0 2025-03-07 11:30:00    0.5  28.972582     8.001064,
                  time  depth  salinity  temperature
 0 2025-03-07 12:30:00    0.5  29.17803     8.009349]

In [21]:
model_df = pd.concat(model_dfs, ignore_index=True)

model_df

,time,depth,salinity,temperature
0,2025-03-07 11:30:00,0.5,28.972582,8.001064
1,2025-03-07 12:30:00,0.5,29.178030,8.009349


In [22]:
result_df = obs_df.join(model_df, rsuffix="_model")

result_df

,time,depth,gridY,gridX,salinity,temperature,time_model,depth_model,salinity_model,temperature_model
0,2025-03-07 11:59:00,0.5,350,250,28.5,7.5,2025-03-07 11:30:00,0.5,28.972582,8.001064
1,2025-03-07 12:30:00,0.5,353,247,28.4,7.6,2025-03-07 12:30:00,0.5,29.178030,8.009349


## Scaling to Many Observations

The data access operations above took about 750 ms per observation on a 1 Gbps network connection with 10 ms latency and very little load on the ERDDAP server.
The server-side processing accounts for about half of that time.
For up to a hundred or so observations,
the technique in this notebook would allow processing to be completed in a minute or two.
For hundreds or thousands of observations we should take advantage of the fact that each request to the server is independent,
making the process quite suitable for parallelization.
That is the subject of another notebook in this directory.